In [5]:
#算法部分，使用pulp内置cdc求解器
import pulp
import numpy as np
import pandas as pd
class ILP():
    def __init__(self,c,T,S):
        #输入说明
        #T:饲料厂产量
        #S:养殖场需求
        #l：距离矩阵
        self.T = T
        self.S = S
        self.row = len(self.T)
        self.col = len(self.S)
        self.c = np.array(c).reshape(self.row,self.col)
        
    def solve(self):
        row = len(self.T) #饲料厂个数
        col = len(self.S) #养殖场个数
        
        prob = pulp.LpProblem('运输配给优化', sense=pulp.LpMinimize)
        
        var = [[pulp.LpVariable(f'x{i}_{j}', cat=pulp.LpBinary) for j in range(col)] for i in range(row)]

        flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]

        prob += pulp.lpDot(flatten(var), self.c.flatten())
        for i in range(row):
            prob += (pulp.lpDot(pulp.lpSum(var[i]),self.S) <= self.T[i])

        for j in range(col):
            prob += (pulp.lpDot([var[i][j] for i in range(row)], [1]*row) == 1)
        prob.solve()
        self.opt_val = round(pulp.value(prob.objective),2)
        self.opt_x = pd.DataFrame([[pulp.value(var[i][j]) for j in range(col)] for i in range(row)])
        #输出说明：
        #opt_val为最低运输距离
        #opt_x为最优运输方式，格式为row（饲料厂数）*col（养殖场数）的0-1矩阵。x[i,j]代表第i间饲料厂是否给第j间养殖场配送饲料
        #部分数据测试及结果
def trans(filename):
    data = pd.read_csv(filename,usecols = ['ffactoryname','ffieldname','fdayscale','fneedfeed'],encoding = 'gbk')
    data.columns = ['饲料厂','产能','养殖场','日需求量']
    养殖场= data[['养殖场','日需求量']].drop_duplicates()
    饲料厂= data[['饲料厂','产能']].drop_duplicates()
    距离矩阵 = pd.read_excel('Data1.xlsx')
    距离矩阵 = 距离矩阵[['FieldName','FeedName','FDistance']]
    距离矩阵 = 距离矩阵.rename(columns = {'FieldName':'养殖场','FeedName' : '饲料厂', 'FDistance' : '距离'})
    data1 = pd.merge(距离矩阵,养殖场,how = 'inner',on = '养殖场')
    data1 = pd.merge(data1,饲料厂,how = 'inner', on = '饲料厂')
    data1 = data1.astype({'产能':'float64'})
    df1 = pd.pivot_table(data1,index=['养殖场'],columns = ['饲料厂'],values = '距离')
    df2 = pd.pivot_table(data1,index=['养殖场'],columns = ['饲料厂'],values = '日需求量')
    df3 = pd.pivot_table(data1,index=['养殖场'],columns = ['饲料厂'],values = '产能')
    l = np.array(df1)
    S = np.array(df2.iloc[:,0])
    T = np.array(df3.iloc[0,:])
    a = ILP(l.T,T,S)
    a.solve()
    S_name = df1.index
    T_name = df1.columns
    a.opt_x.columns = S_name
    a.opt_x.index = T_name
    res = pd.DataFrame([list(a.opt_x.index[np.where(a.opt_x == 1)[0]])]+[a.opt_x.columns[np.where(a.opt_x == 1)[1]].to_list()],index = ['饲料厂','养殖场'])
    res = pd.DataFrame(res.values.T, index=res.columns, columns=res.index)
    res = pd.merge(res,data1,on = ['饲料厂','养殖场'])
    return res


In [6]:
trans('4月份供料信息.csv')

,饲料厂,养殖场,距离,日需求量,产能
0,双辽饲料机组2,万荣2场,1735188,37.35,272.73
1,双辽饲料机组2,万荣2场,1735188,46.54,272.73
2,双辽饲料机组2,万荣4场,1729466,97.63,272.73
3,双辽饲料机组2,万荣4场,1729466,80.00,272.73
4,双辽饲料机组2,万荣5场,1726054,30.06,272.73
5,双辽饲料机组2,万荣5场,1726054,35.80,272.73
6,双辽饲料机组2,上蔡1场,1694021,112.00,272.73
7,双辽饲料机组2,上蔡3场,1717544,36.27,272.73
8,双辽饲料机组2,上蔡3场,1717544,80.00,272.73
9,双辽饲料机组2,东明16场,1504538,140.99,272.73
